In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import exp
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
%matplotlib inline

In [2]:
#Spec = np.loadtxt('HE1108-2813_ASCII', unpack=False)
hdu = fits.open('Radio Loud Table.fits')
data_tot = hdu[1].data
data= data_tot[data_tot['L'] == 1]

print (len(data))

9863


In [3]:
flux_OIII_5007 = data['OIII_5007_FLUX']
flux_Hb = data['H_BETA_FLUX']
flux_Ha = data['H_ALPHA_FLUX']
flux_NII_6584 = data['NII_6584_FLUX']
flux_SII_6717 = data['SII_6717_FLUX']
flux_SII_6731 = data['SII_6731_FLUX']
z = data['z_1']
length = len(flux_OIII_5007)
ra = data['RA']
dec = data['DEC']

In [4]:
quantity_1 = np.log10(flux_OIII_5007/flux_Hb) - np.log10(flux_NII_6584/flux_Ha)
quantity_2 = np.log10(flux_OIII_5007/flux_Hb) - np.log10((flux_SII_6717+flux_SII_6731)/flux_Ha)
SN = data['OIII_5007_FLUX']/data['OIII_5007_FLUX_ERR']

/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/anaconda3/lib/

In [5]:
for i in range (length):
    if quantity_1[i] > 0 and quantity_1[i] < 0.7 and quantity_2[i] > 0 and quantity_2[i] < 0.9:
        print ('%s,%s,%s'%(data['Plate'][i],data['MJD_1'][i],data['Fiber'][i]))
#        print ('%s,%s,%s,%s'%(data['Plate'][i],data['MJD_1'][i],data['Fiber'][i],SN[i]))
        popt = [data['Plate'][i],data['MJD_1'][i],data['Fiber'][i],SN[i],ra[i],dec[i]]
        column_names={'PLATEID':0,'MJD':1,'FIBERID':2,'OIII_5007_SN':3,'RA':4,'DEC':5}
        columns=[]
        for key in column_names.keys():
            columns.append(fits.Column(name=key,format='E',array=[popt[column_names[key]]]))
            coldefs = fits.ColDefs(columns)
            hdu = fits.BinTableHDU.from_columns(coldefs)
            hdu.writeto('LERG/%s_%s_%s_LERG.fits'%(data['Plate'][i],data['MJD_1'][i],data['Fiber'][i]),overwrite=True)
    else:
        pass
    len(data)

266,51602,26
267,51608,34
267,51608,97
267,51608,260
267,51608,297
267,51608,512
268,51633,38
268,51633,271
269,51910,171
269,51910,289
270,51909,125
271,51883,93
271,51883,494
272,51941,437
272,51941,442
273,51957,633
274,51913,218
274,51913,430
275,51910,291
275,51910,447
277,51908,23
277,51908,168
278,51900,184
278,51900,254
280,51612,244
281,51614,206
281,51614,616
283,51959,129
283,51959,420
284,51662,114
286,51999,16
287,52023,187
289,51990,138
290,51941,44
290,51941,89
291,51660,594
292,51609,369
292,51609,519
295,51985,557
296,51665,24
296,51665,122
298,51955,353
299,51671,31
299,51671,125
300,51666,209
300,51666,250
300,51666,391
301,51641,452
302,51688,377
304,51609,57
304,51609,319
304,51609,352
305,51613,204
307,51663,255
308,51662,285
311,51665,158
311,51665,172
312,51689,3
312,51689,318
312,51689,471
313,51673,248
313,51673,616
314,51641,222
315,51663,357
326,52375,164
328,52282,63
328,52282,112
328,52282,477
329,52056,173
329,52056,455
330,52370,468
330,52370,533
331,523

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import curve_fit
from numpy import exp
from scipy import ndimage
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

from shutil import copyfile

def script2(input_filename,output_filename):
    dir_name = 'LERG'   
    os.chdir(dir_name)
    infiles = sorted(glob.glob(input_filename)) # './*source.fits'
    tabledict = {}
  
    for i, file in enumerate(infiles):
        hdulist = fits.open(file)
        bintab = hdulist[1].data
        table = Table(bintab)
        name = table['OIII_5007_SN'][0]  
        tabledict[name] = table

    tab = vstack(list(tabledict.values()))
    df = tab.to_pandas()
    df = df.sort_values(by='OIII_5007_SN', ascending=True)
    dtab = Table.from_pandas(df)
    dtab.write(output_filename, format='fits', overwrite=True)
    
script2(input_filename='./*LERG.fits',output_filename='LERG_cord.fits')


In [2]:
from voronoi_bin import *